# **SchnetPack Ensemble Calculator**

## Preparing and loading your data

This section prepares the computational environment by importing essential libraries for:

- **Atomistic simulations** using ASE (Atomic Simulation Environment).
- **Machine learning potentials** via SchNetPack, including ensemble support for uncertainty estimation.
- **MD17 Dataset**, a standard benchmark for ML-based potential energy surfaces.
- **Molecular dynamics (MD)** tools like Langevin dynamics and thermal velocity initialization using Maxwell-Boltzmann distributions.
- **Optimization and analysis utilities**, such as LBFGS optimizer and plotting with `matplotlib`.

These imports provide the tools to perform uncertainty-aware molecular dynamics simulations with an ensemble of neural network models, enabling deeper insight into the reliability of predicted forces and energies across the MD trajectory.


In [ ]:
from ase import Atoms
import schnetpack as spk
from schnetpack.interfaces.ase_interface import SpkEnsembleCalculator, AbsoluteUncertainty, RelativeUncertainty
import schnetpack.transform as trn
from schnetpack.datasets import MD17
import os
import numpy as np
from tqdm import tqdm
from ase import units
from ase.md.velocitydistribution import MaxwellBoltzmannDistribution
from ase.md.langevin import Langevin
import matplotlib.pyplot as plt
import random
from ase.io import read
from ase.optimize.lbfgs import LBFGS

## Predefined datasets

SchNetPack supports several benchmark datasets that can be used without preparation.
Each one can be accessed using a corresponding class that inherits from `AtomsDataModule` (a specialized PyTorchLightning `DataModule`), which supports automatic download, conversion and partitioning. Here, we show how to use these data sets at the example of the QM17 benchmark.

First, we have to import the dataset class and instantiate it. This will automatically download the data to the specified location.

In [ ]:
## Load the dataset
ensembletut = './ensembletut'
if not os.path.exists(ensembletut):
    os.makedirs(ensembletut)

ethanol_data = MD17(
    os.path.join(ensembletut,'ethanol.db'),
    molecule='ethanol',
    batch_size=10,
    num_train=1000,
    num_val=1000,
    transforms=[
        trn.ASENeighborList(cutoff=5.),
        trn.RemoveOffsets(MD17.energy, remove_mean=True, remove_atomrefs=False),
        trn.CastTo32()
    ],
    num_workers=0,
    pin_memory=False, 
)
ethanol_data.prepare_data()

- We initialize the dataset with custom `transforms` tailored for energy-conserving, differentiable models:
  - `ASENeighborList` constructs pairwise neighbor lists, ensuring SchNet’s locality assumptions are respected.
  - `RemoveOffsets` is applied with `remove_mean=True` but `remove_atomrefs=False`, aligning energy targets without removing per-element references—useful for force-learning scenarios where relative energy differences matter more than absolute values.
  - `CastTo32` ensures all model inputs remain in `float32`, balancing numerical stability and performance on GPU/TPU hardware.
- A relatively small training set is used (1000 samples), mimicking real-world data limitations and emphasizing the need for uncertainty-aware modeling (via ensemble predictions).
- The `.prepare_data()` call handles caching and preprocessing efficiently for downstream training and evaluation workflows.

This configuration lays the groundwork for benchmarking SchNetPack’s ensemble and uncertainty tools in data-efficient regimes.

In [ ]:
ats = read("../../examples/howtos/ensembletut/ethanol.db", index=":")
index = 1
atoms = ats[index]


## Ensemble Interface to ASE

We specify a list of pre-trained SchNet models trained on ethanol data. These models constitute the **ensemble**, which will be used to serve as a testbed for SchNetPack’s ensemble-enabled `SpkEnsembleCalculator`, which supports integrated uncertainty quantification.

In [ ]:
model_path_list = ['../../tests/testdata/md_ethanol.model',
                   '../../tests/testdata/md_ethanol_2.model']

### ⚖️ Creating an Ensemble Calculator with Uncertainty Quantification

In this section, we instantiate two different uncertainty estimators:
- `AbsoluteUncertainty` captures **epistemic uncertainty** by evaluating standard deviations of ensemble predictions (in energy, forces, or stress).
- `RelativeUncertainty` normalizes this by the predicted mean, which is useful when evaluating models across a range of energy/force magnitudes.

By passing both uncertainty functions to the `SpkEnsembleCalculator`, we enable simultaneous access to **multiple uncertainty metrics** during molecular simulations.

The `SpkEnsembleCalculator` is designed to be a drop-in replacement for any standard ASE calculator, while internally averaging predictions from multiple models and computing uncertainty over the ensemble. The `energy_unit` and `position_unit` are provided in MD17-specific units (kcal/mol and Å), and will automatically be converted to ASE’s standard internal units (eV, eV/Å), ensuring consistency across simulations.

The `ASENeighborList` transform defines the spatial cutoff for atom-wise interactions — a critical parameter that influences the model's perception of local chemical environments.


In [ ]:
uncertainty_abs = AbsoluteUncertainty(energy_weight=0.5,force_weight=1.0)
uncertainty_rel = RelativeUncertainty(energy_weight=1.0, force_weight=2.0)

uncertainty = [uncertainty_abs, uncertainty_rel]

ensemble_calculator = SpkEnsembleCalculator(
    models=model_path_list,
    neighbor_list=trn.ASENeighborList(cutoff=5.0),
    energy_key=MD17.energy,
    force_key=MD17.forces,
    energy_unit="kcal/mol",
    position_unit="Ang",
    uncertainty_fn=uncertainty)

Assign the ensemble calculator `ensemble_calculator` to the atoms object

In [ ]:
atoms.calc = ensemble_calculator

 🔮 Prediction Output:
- ⚡ Energy: Total potential energy of the atomic system.
- 🔧 Forces: Atomic forces (gradients of energy) for optimization or dynamics.
- 📊 Uncertainty: Estimation of model prediction uncertainty from the ensemble.

In [ ]:
print("Prediction:")
print("energy:", atoms.get_total_energy())
print("forces:", atoms.get_forces())
print("uncertainty:", ensemble_calculator.get_uncertainty(atoms))

# Structure Optimization

🏗️ **Load and Distort Molecular Structure:**
- 📂 **Input file**: "../../tests/testdata/md_ethanol.xyz" (initial structure).
- 🧬 **Read the structure** using `read()` and extract atomic positions.
- 🔄 **Distort the atomic positions** randomly within a small range (95% to 105% of original).
- 🧪 **Create a new molecule** with distorted positions using `Atoms()` constructor.


In [ ]:
input_structure_file = "../../tests/testdata/md_ethanol.xyz"
# load initial structure
mol = read(input_structure_file)
pos = mol.get_positions()

# distort the structure
for n in range(pos.shape[0]):
    pos[n] = pos[n] * random.uniform(0.95,1.05)
    
molecule = Atoms(positions=pos, numbers=mol.get_atomic_numbers())


⚙️ **Set Up Optimization:**
- 🧑‍🔬 **Optimizer**: Using the **LBFGS** algorithm for geometry optimization of the molecule.
- 🔬 **Calculator**: Assigning the **ensemble calculator** (`ensemble_calculator`) to the molecule for energy/force predictions during optimization.

In [ ]:
optimizer = LBFGS(molecule)
molecule.calc = ensemble_calculator

🔄 **Optimization Loop with Uncertainty Tracking:**
- 🧑‍🔬 **Optimizer**: Run the optimization using the LBFGS algorithm with a force tolerance of 0.01 and a maximum of 100 steps.
- 📊 **Uncertainty Tracking**: After each optimization step, the uncertainty of the energy prediction is appended to the `uncertainties` list, providing insight into model confidence during the process.

In [ ]:
uncertainties = []  

for _ in optimizer.irun(fmax=0.01, steps=100):
    uncertainties.append(ensemble_calculator.get_uncertainty(molecule))

- **Absolute** and **Relative Uncertainty** values are extracted from the optimization steps.
- Plot both **absolute** and **relative** uncertainties against the optimization steps to visualize how uncertainty changes during the process.

In [ ]:
# Extract individual uncertainty types
abs_vals = [d["AbsoluteUncertainty"] for d in uncertainties]
rel_vals = [d["RelativeUncertainty"] for d in uncertainties]

# Now plot both
plt.figure(figsize=(8, 6))
plt.plot(abs_vals, label="Absolute Uncertainty", marker='o')
plt.plot(rel_vals, label="Relative Uncertainty", marker='x')
plt.xlabel("Optimization Step")
plt.ylabel("Uncertainty")
plt.title("Uncertainty during Optimization")
plt.legend()
plt.grid(True)
plt.show()


Since we're using an ensemble of models, we can now estimate the uncertainty in our predictions during the optimization process.
While the absolute uncertainty remains low and stable throughout, the relative uncertainty shows sharp fluctuations at specific steps.
This indicates that although the model is generally confident in its predictions, there are certain configurations where the predictions become less reliable relative to their magnitude.
These fluctuations can serve as useful indicators of difficult or out-of-distribution structures during optimization.

# Molecular Dynamics With Annealing Temperature

- The **SpkEnsembleCalculator** is initialized with multiple models from `model_path_list`, a neighbor list (`ASENeighborList`) with a 5.0 Å cutoff, and keys for energy and forces (`MD17.energy` and `MD17.forces`).
- Energy and positions are handled in "kcal/mol" and "Ang", respectively.
- **Uncertainty Function**: The previously defined `uncertainty_abs` function is passed for uncertainty calculation.

🔎 **Note**: The `uncertainty_fn` can be passed as either a **single** uncertainty function or as a **list** of uncertainty functions, depending on the desired level of complexity.


In [ ]:
uncertainty_abs = AbsoluteUncertainty(energy_weight=0.5,force_weight=1.0)

abs_ensemble_calculator = SpkEnsembleCalculator(
    models=model_path_list,
    neighbor_list=trn.ASENeighborList(cutoff=5.0),
    energy_key=MD17.energy,
    force_key=MD17.forces,
    energy_unit="kcal/mol",
    position_unit="Ang",
    uncertainty_fn=uncertainty_abs)

In [ ]:
atoms.calc = ensemble_calculator

## Temperature Annealing and Langevin Dynamics with SchNetPack

In this simulation, we apply **temperature annealing** and **Langevin dynamics** to model the behavior of ethanol under varying temperatures, optimizing molecular configurations and simulating thermal effects.

### Annealing

Annealing gradually cools the system from a high initial temperature ($T_{\text{start}}$) to a lower final temperature ($T_{\text{end}}$) over time. The temperature is decayed exponentially:

$$
T_{\text{annealed}}(t) = T_{\text{start}} \times \exp\left(-\frac{t}{n_{\text{steps}}} \times \ln\left(\frac{T_{\text{start}}}{T_{\text{end}}}\right)\right)
$$

This cooling schedule helps the system avoid local minima and settle into a more stable configuration.

### Langevin Dynamics

Langevin dynamics includes both deterministic forces and stochastic noise to simulate the interaction with a thermal bath. The equation of motion for atom $i$ is:

$$
m_i \frac{d^2 \mathbf{r}_i}{dt^2} = \mathbf{F}_i - \gamma m_i \frac{d \mathbf{r}_i}{dt} + \sqrt{2 \gamma k_B T} \xi_i(t)
$$

Where:
- $\mathbf{F}_i$ is the force acting on atom $i$,
- $\gamma$ is the friction coefficient,
- $m_i$ is the mass of the atom,
- $T$ is the temperature, and
- $\xi_i(t)$ is a random Gaussian noise term.

This term controls the system's energy dissipation and allows thermal fluctuations, mimicking a heat bath.

### Uncertainty Estimation with Machine Learning Potentials

SchNetPack uses machine learning models to predict energy, forces, and uncertainties. The uncertainty is calculated from the variation between different models in the ensemble, giving a measure of confidence in the predictions.

By combining **annealing**, **Langevin dynamics**, and **uncertainty estimation**, we can simulate molecular systems with thermal fluctuations and optimize configurations. The use of machine learning potentials enables more accurate and efficient simulations, particularly for large systems where classical methods may be less reliable.


In [ ]:
# Annealing parameters
temperature_start = 500  
temperature_end = 50    
n_steps = 1000          
sampling_interval = 10  
step_size = 0.5         

# Cooling schedule function: Exponential decay of temperature
def get_annealed_temperature(step, n_steps, start_temp, end_temp):
    return start_temp * np.exp(-step / n_steps * np.log(start_temp / end_temp))

MaxwellBoltzmannDistribution(molecule, temperature_K=temperature_start)

dyn = Langevin(
    molecule, 
    timestep=step_size * units.fs, 
    temperature_K=temperature_start,
    friction=0.001 / units.fs
)

ats_traj = []
uncertainties = []
e_tmp = []

for step in tqdm(range(n_steps // sampling_interval)):
    annealed_temperature = get_annealed_temperature(step, n_steps, temperature_start, temperature_end)

    dyn.temperature_K = annealed_temperature

    dyn.run(sampling_interval)

    e_tmp.append(molecule.get_temperature())
    uncertainties.append(abs_ensemble_calculator.get_uncertainty(molecule))
    
    ats_traj.append(molecule.copy())

    print(f"Step {step * sampling_interval}, Annealed Temp: {annealed_temperature:.2f} K")

This code generates a dual-axis plot showing the evolution of uncertainty (blue) and temperature (red) during molecular dynamics, with uncertainty tracked on the left y-axis and temperature on the right. The plot helps visualize the system's reliability and thermal behavior over time.

In [ ]:
fig, ax1 = plt.subplots(figsize=(8, 6))

ax1.plot(uncertainties, marker='o', color='blue', label='Uncertainty')
ax1.set_xlabel("MD Step")
ax1.set_ylabel("Uncertainty", color='blue')
ax1.tick_params(axis='y', labelcolor='blue')

ax2 = ax1.twinx()
ax2.plot(e_tmp, marker='x', color='red', label='Temperature')
ax2.set_ylabel("Temperature (K)", color='red')
ax2.tick_params(axis='y', labelcolor='red')

plt.title("Molecular Dynamics: Uncertainty and Temperature Profile")
ax1.grid(True)

lines_1, labels_1 = ax1.get_legend_handles_labels()
lines_2, labels_2 = ax2.get_legend_handles_labels()
ax1.legend(lines_1 + lines_2, labels_1 + labels_2, loc='upper right')

plt.tight_layout()
plt.show()

This annealing MD simulation cools the system from 500 K to 50 K via an exponential decay schedule. The temperature profile (red) reflects this trend with expected thermal fluctuations.

Throughout the trajectory, the ensemble model's uncertainty (blue) remains stable despite the changing thermodynamic conditions. This indicates robust generalization of the model across varying configurations, ensuring reliable predictions even as the system explores lower-energy states.